This script utilizes the OpenAI API (GPT-3.5-turbo model) LLM to perform NLP in the form of text classification. I make the API go through the entire "Narrative" column, extracting the text for each entry to determine the cause of the aircraft incident. All entries will fall in these specific categories:
- Pilot error
- Mechanical failure
- Weather
- ATC error
- Sabotage
- Other
- Unknown

I have provided category descriptions to give the LLM more context to improve the accruacy of the text classification. After categorizing the text data, I put the results into a list and go to the "cleaned_asndb.csv" file and append the new "Primary causes" column into it.

In [1]:
import pandas as pd
import time
from tqdm import tqdm
%pip install openai==0.28
import openai
#print(openai.__version__)

# Set up with my key
openai.api_key = "MY_API_KEY" ##<< API KEY HIDDEN FOR PRIVACY


#Load the narrative.csv file created specific to read the narrative texts
narrative_df = pd.read_csv('narrative.csv')
narrative_df['Narrative'] = narrative_df['Narrative'].fillna('')

# GIVE MORE INFO TO GPT-3.5 LLM to improve NLP
category_descriptions = {
    'Pilot error': """
    Errors in judgment, decision-making, or action by the flight crew. 
    This includes failure to follow procedures and protocol, mishandling of aircraft controls, 
    miscommunication, navigation errors, spatial disorientation, fatigue-related mistakes, or inadequate preparation.
    """,
    'Mechanical failure': """
    Malfunctions or failures of aircraft systems, engines, or components. 
    This includes issues with engines, landing gear, hydraulics, electrical and computer systems, 
    or structural failures not caused by external factors.
    """,
    'Weather': """
    Adverse weather conditions. This includes 
    thunderstorms, icing, turbulence, low visibility, fog, wind shear, or any 
    weather phenomenon that caused aircraft to crash.
    """,
    'ATC error': """
    Errors caused by air traffic control miscommunication or mistakes. ATC must be the primary cause.
    """,
    'Sabotage': """
    Intentional actions to damage or compromise the safety of the aircraft. 
    This includes terrorist acts, hijackings, explosives, explosions, bombings, pilot suicide, or any deliberate 
    interference with the aircraft's systems or operation.
    """,
    'Other': """
    Causes that don't fit into the above categories. This might include 
    bird strikes, ground collisions, ground crew issues, heart attacks, a single pilot/passenger random death, or rare events like space debris impact.
    """,
    'Unknown': """
    Cases where the cause cannot be determined or is not clearly stated 
    in the available information, disappeared aircraft not found
    """
}

category_descriptions_str = "\n".join([f"{k}: {v.strip()}" for k, v in category_descriptions.items()])

# PROMPT
def classify_narrative(narrative):
    prompt = f"""Classify the following aircraft incident narrative into one of these categories, do not give an explanation just the category name, the only values outputted should be part of this list:
{category_descriptions_str}

Narrative: {narrative}

Category:"""
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    
    category = response['choices'][0]['message']['content'].strip()
    return category

primary_causes = []

# Loop through each narrative and classify it
for narrative in tqdm(narrative_df['Narrative'], desc="Classifying Narratives"):
    primary_cause = classify_narrative(narrative)
    primary_causes.append(primary_cause)
    time.sleep(1)  # Add a delay to avoid rate limiting/ API has certain requests per minute
   
narrative_df['Primary cause'] = primary_causes

# Load the original 'final_asndb.csv' file
final_df = pd.read_csv('final_asndb.csv')

# Make sure rows match with original narrative.csv df
if len(final_df) == len(narrative_df):
    # Append primary cause column
    final_df['Primary cause'] = narrative_df['Primary cause']
    
    final_df.to_csv('final_asndb.csv', index=False)
    print("text classification complete. The final CSV has been saved as 'final_asndb.csv'.")
else:
    print("Error: The # of rows in 'final_asndb.csv' and 'narrative.csv' doesn't match.")

Note: you may need to restart the kernel to use updated packages.


Classifying Narratives: 100%|██████████| 693/693 [17:33<00:00,  1.52s/it]

Classification complete. The final CSV has been saved as 'final_asndb.csv'.


I perform further cleaning of this .csv file into a final form to be used by my PostgreSQL database. This is the .csv that is extracted into the SQL database, allowing queries from the frontend of my webpage.

In [ ]:
import pandas as pd
df = pd.read_csv('final_asndb.csv')

# Convert date column, should be MM-DD-YYYY to be compatible with SQL "date" classification
df['Date'] = pd.to_datetime(df['Date'], format='%A %d %B %Y').dt.strftime('%Y-%m-%d')

# Make sure time column is in HH:MM:SS format
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce').dt.strftime('%H:%M:%S')

# Save the cleaned CSV
df.to_csv('final_cleaned_asndb.csv', index=False)
